## Parse patient zero results - proximity

In [1]:
import numpy as np
import numba as nb
import pandas as pd

import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../../src')
import networkx as nx

In [2]:
import io_m.libsaving as libsaving
import io_m.io_utils as io_utils

from importlib import reload

In [3]:
from epigen import epidemy_gen

In [4]:
from epigen.base import EpInstance

In [5]:
type_graph="i_bird"
N=10
d=3
height=3
lam=0.5
mu=1e-6
t_limit=12
scale=2
#gamma=1e-3
#path_contacts="../patient_zero/work/work_13_contacts.npz"
#small_lambda_limit=0

gamma=7e-5
small_lambda_limit=300
path_contacts="work_13_contacts.npz"
path_contacts="i_bird_contacts.npz"

#path_contacts="i_bird_contacts.npz"

p_edge=1
nsrc=1


In [6]:
seeds=range(1,2)

In [7]:
INSTANCES = {s:EpInstance(type_graph, N, d, t_limit, 
                                    lam, mu, s, float(p_edge), 
                                    n_source=nsrc) for s in seeds}

In [8]:
str(INSTANCES[1])

'i_bird_n_10_d_3_tlim_12_lam_0.5_mu_1e-06_s_1_pe_1.0'

In [9]:
extra_gen = {"h":height,
            "scale":scale}

extra_gen.update({"gamma":gamma, 
                  "small_lambda_limit":small_lambda_limit, 
                  "path_contacts": path_contacts})

In [10]:
start_conf=0
num_conf=1000

In [11]:
reload(epidemy_gen)
sources=set([50])
DATA_ALL = {s: 
            epidemy_gen.epidemy_gen_epinstance(inst, 
                                               lim_infected=1, num_conf=num_conf,
                                               extra_gen=extra_gen, 
                                               verbose=False,
                                              num_sources=nsrc,
                                              sources=sources) 
            for s, inst in INSTANCES.items()}


number of contacts: 14010
Lim infected: 1, Lim max infected: 327
Num sources:  1
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources {50}
 Sources 

In [12]:
from annfore.net import nn_sir_path_obs
from annfore.utils.graph import find_neighs
from annfore.models import common as en_common
from annfore.models import sir_model_N_obs
from annfore.learn.losses import loss_fn_coeff
from annfore.learn.train import train_beta, train_saving_marginals, make_training_step_local
from annfore.learn.opt import make_opt

from annfore.learn.train_params import opt_param_init, train_beta_params
from annfore.learn.train_params import learn_lamb_mu, learn_gamma_mu
from annfore.learn.l_utils import make_beta_sequence_three as make_beta_seq


In [13]:
DATA_ALL.keys()

dict_keys([1])

In [14]:
# func_layers = [1, 1]
contacts = DATA_ALL[1]["contacts"]
N = int(max(contacts[:, 1]) + 1)
mu = 0
p_source = 1/N
neighs = find_neighs(contacts,N=N,only_minor=True, next_near_neigh=True)

In [15]:
instance_num = 0
last_obs = DATA_ALL[1]["test"][instance_num][1]
obs_list_net = []
obs_list = []
for i, s in enumerate(last_obs):
    obs_list_net.append([i,s,t_limit])
    obs_list.append([i,t_limit,s])

In [19]:
device = "cuda"
func_layers = [1,1]
my_net = nn_sir_path_obs.SIRPathColdObs(neighs,
                    t_limit+1,
                    obs_list=obs_list_net,
                    hidden_layer_spec=func_layers,
                    device = device,
                    bias=True,
                    )

[0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [20]:
p_mc = np.exp(np.log(p_source) - t_limit)
p_obs = p_mc
model = sir_model_N_obs.SirModel(contacts,
                        mu = mu,
                        device = device,
                            p_source=p_source,
                            p_obs=p_obs,
                            p_w=p_mc)
model.set_obs(obs_list_net)

In [21]:
num_samples = 1000
lr=1e-3
optimizer = []
for i in range(N):
    if len(my_net.params_i[i]):
        optimizer.append(make_opt(my_net.params_i[i], lr=lr))
    else:
        optimizer.append([])
t_obs = t_limit
n_beta_steps = 10000
betas = np.linspace(0,1,n_beta_steps+1)[:-1]
name_file_instance = "i_bird" + "_" + str(instance_num)


In [22]:
loss_fun = loss_fn_coeff
extra_args = None

results = train_beta(my_net, optimizer,
                        model, name_file_instance,
                        loss_fun, t_obs,
                        num_samples=num_samples,
                        train_step = make_training_step_local,
                        betas=betas, save_every=200,
                        loss_extra_args=extra_args,
                        save_net=True)

 200 beta: 0.0200, loss: 7358.658, std: 98.376, ener: 7736.53, max_grad = 0.0, count_zero_pw = 64643.00, num_I = 148.000, num_R = 0.0000 (T_obs=12) -- took   1399 ms: sample: 277, log_prob: 0, trans_sample: 0, energy: 678, loss: 0, optim_step: 441, stats: 1 -- source: { 245:0.117, 272:0.109, 237:0.104, 323:0.102, 222:0.102,  18:0.101, 281:0.101, 316:0.100, 309:0.100, 184:0.099, 283:0.098, 246:0.098, 274:0.098,  56:0.097,  86:0.096, 255:0.096, 198:0.095, 186:0.095, 235:0.095, 182:0.094}    - remain time:  3h 47m 22 sdict_keys(['step', 'beta', 'energy', 'std_energy', 'loss', 'loss_std', 'S', 'I', 'R', 't_obs', 'num_source', 'sources', 'max_grad', 'num_zero_pw', 'N', 'T', 'p_source', 'p_sus', 'p_obs', 'p_w', 'mu', 'times'])
 400 beta: 0.0400, loss: 7193.625, std: 93.505, ener: 7566.56, max_grad = 0.0, count_zero_pw = 56547.00, num_I = 148.000, num_R = 0.0000 (T_obs=12) -- took   1483 ms: sample: 240, log_prob: 0, trans_sample: 0, energy: 622, loss: 0, optim_step: 617, stats: 2 -- source: 

In [16]:
import torch

In [17]:
net_load = torch.load("./i_bird_0.pt")

In [18]:
num_sample = 1000
ss, h = net_load.sample(num_sample)

In [19]:
torch.save(ss, "samples_ibird.torch")

In [129]:
sample_free = []
for epid in range(len(DATA_ALL[1]["epidemy"]))[:num_sample]:
    sample_free.append(DATA_ALL[1]["epidemy"][epid][0][0])

In [130]:
sample_free = torch.Tensor(sample_free)

In [131]:
sample_free[sample_free == float("Inf")] = 13

In [132]:
sample_free

tensor([[13., 13.,  2.,  ...,  8., 10., 13.],
        [13., 13., 13.,  ..., 13., 10., 13.],
        [13., 13.,  3.,  ...,  8., 10., 13.],
        ...,
        [13., 13., 13.,  ..., 13., 13., 13.],
        [ 2., 13.,  3.,  ..., 13., 10., 13.],
        [13., 13., 13.,  ...,  8., 13., 11.]])

In [133]:
clone_ss = ss.clone()

In [134]:
clone_ss[:,:,0] = sample_free[:]+1
clone_ss[clone_ss >13] = 13

In [135]:
clone_ss

tensor([[[13, 13],
         [13, 13],
         [ 3, 13],
         ...,
         [ 9, 13],
         [11, 13],
         [13, 13]],

        [[13, 13],
         [13, 13],
         [13, 13],
         ...,
         [13, 13],
         [11, 13],
         [13, 13]],

        [[13, 13],
         [13, 13],
         [ 4, 13],
         ...,
         [ 9, 13],
         [11, 13],
         [13, 13]],

        ...,

        [[13, 13],
         [13, 13],
         [13, 13],
         ...,
         [13, 13],
         [13, 13],
         [13, 13]],

        [[ 3, 13],
         [13, 13],
         [ 4, 13],
         ...,
         [13, 13],
         [11, 13],
         [13, 13]],

        [[13, 13],
         [13, 13],
         [13, 13],
         ...,
         [ 9, 13],
         [13, 13],
         [12, 13]]], device='cuda:0')

In [136]:
model_free = sir_model_N_obs.SirModel(contacts,
                        mu = 0,
                        device = device,
                            p_source=1,
                            p_obs=1,
                            p_w=1e-30)

In [141]:
logp_net, _,_,_ = model_free.energy_separated(ss)

In [142]:
logp_sim, _,_,_ = model_free.energy_separated(clone_ss)

In [143]:
logp_net.mean()

tensor(229.1972, device='cuda:0')

In [144]:
logp_sim.mean()

tensor(175.8102, device='cuda:0')